In [1]:
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Multiply,GlobalAveragePooling2D, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Lambda
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K

Using TensorFlow backend.


In [0]:
from google.colab import files
uploaded=files.upload()

In [0]:
!wget "https://www.cs.rochester.edu/u/qyou/DeepSent/Agg_AMT_Candidates.zip"

In [0]:
!unzip Agg_AMT_Candidates.zip

In [0]:
import os
import tensorflow as tf
from tensorflow import keras
from keras.utils import Sequence
import pandas as pd
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, ImageDataGenerator
import math
import random
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import sklearn
from sklearn.model_selection import train_test_split
SEED = 2001
from keras import Model

In [0]:
df = pd.read_csv('image3.csv')
df.head(10)

,image_name,label
0,717364737_466173837713743872_Bkkoyj3IcAAP3y7.jpg,1
1,2544810430_466243949704019968_BngaM4WIMAAZYlo.jpg,1
2,334460609_463516330898231296_Bm65hTHCIAA-Z6w.jpg,0
3,276867046_466225763222441984_BnhUI4DCEAE-yNW.jpg,1
4,469723431_463453458297663488_Becy2ktCcAAi3gd.jpg,1
5,1909939033_466237456933584897_Bng7oBrIYAEpXs2.jpg,0
6,16291163_467436465845071872_BnyqAOGCIAA2wlg.jpg,0
7,74552359_463481790809006081_Bm6RUxnIgAE4qo7.jpg,0
8,1861587818_466222256754933761_Bjgp1wGCYAAt8ky.jpg,0
9,356198571_463874235048263680_Bm_kcgDCQAAnTCi.jpg,1


In [0]:
df['label'] = df.label.astype(str)

In [0]:
from sklearn.model_selection import StratifiedKFold
from keras.layers import GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint

In [0]:
x=df.image_name
y=df.label

In [0]:
def res_conv(X, filters, base, s):
    
    name_base = base + '/branch'
    
    F1, F2, F3 = filters

    ##### Branch1 is the main path and Branch2 is the shortcut path #####
    
    X_shortcut = X
    
    ##### Branch1 #####
    # First component of Branch1 
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_1')(X)
    X= Activation('relu', name=name_base + '1/relu_1')(X)
    X = Conv2D(filters=F1, kernel_size=(1,1), strides=(1,1), padding='valid', name=name_base + '1/conv_1', kernel_initializer=glorot_uniform(seed=0))(X)

    # Second component of Branch1
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_2')(X)
    X = Activation('relu', name=name_base + '1/relu_2')(X)
    X = Conv2D(filters=F2, kernel_size=(3,3), strides=(s,s), padding='same', name=name_base + '1/conv_2', kernel_initializer=glorot_uniform(seed=0))(X)
    
    # Third component of Branch1
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_3')(X)
    X = Activation('relu', name=name_base + '1/relu_3')(X)
    X = Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding='valid', name=name_base + '1/conv_3', kernel_initializer=glorot_uniform(seed=0))(X)
    
    ##### Branch2 ####
    X_shortcut = BatchNormalization(axis=-1, name=name_base + '2/bn_1')(X_shortcut)
    X_shortcut= Activation('relu', name=name_base + '2/relu_1')(X_shortcut)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1,1), strides=(s,s), padding='valid', name=name_base + '2/conv_1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    
    # Final step: Add Branch1 and Branch2
    X = Add(name=base + '/Add')([X, X_shortcut])

    return X

In [0]:
def res_identity(X, filters, base):
    
    name_base = base + '/branch'
    
    F1, F2, F3 = filters

    ##### Branch1 is the main path and Branch2 is the shortcut path #####
    
    X_shortcut = X
    
    ##### Branch1 #####
    # First component of Branch1 
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_1')(X)
    Shortcut= Activation('relu', name=name_base + '1/relu_1')(X)
    X = Conv2D(filters=F1, kernel_size=(1,1), strides=(1,1), padding='valid', name=name_base + '1/conv_1', kernel_initializer=glorot_uniform(seed=0))(Shortcut)

    # Second component of Branch1
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_2')(X)
    X = Activation('relu', name=name_base + '1/relu_2')(X)
    X = Conv2D(filters=F2, kernel_size=(3,3), strides=(1,1), padding='same', name=name_base + '1/conv_2', kernel_initializer=glorot_uniform(seed=0))(X)
    
    # Third component of Branch1
    X = BatchNormalization(axis=-1, name=name_base + '1/bn_3')(X)
    X = Activation('relu', name=name_base + '1/relu_3')(X)
    X = Conv2D(filters=F3, kernel_size=(1,1), strides=(1,1), padding='valid', name=name_base + '1/conv_3', kernel_initializer=glorot_uniform(seed=0))(X)    
    
    # Final step: Add Branch1 and the original Input itself
    X = Add(name=base + '/Add')([X, X_shortcut])

    return X

In [0]:
def Trunk_block(X, F, base):
    
    name_base = base
    
    X = res_identity(X, F, name_base + '/Residual_id_1')
    X = res_identity(X, F, name_base + '/Residual_id_2')
    
    return X

In [0]:
def interpolation(input_tensor, ref_tensor,name): # resizes input_tensor wrt. ref_tensor
    H, W = ref_tensor.get_shape()[1], ref_tensor.get_shape()[2]
    return tf.image.resize_nearest_neighbor(input_tensor, [H.value, W.value],name=name)

In [0]:
def Attention_1(X, filters, base):
    
    F1, F2, F3 = filters
    
    name_base = base
    
    X = res_identity(X, filters, name_base+ '/Pre_Residual_id')
    
    X_Trunk = Trunk_block(X, filters, name_base+ '/Trunk')
    
    X = MaxPooling2D((3,3), strides=(2,2), padding='same', name=name_base+ '/Mask/pool_3')(X)
    
    X = res_identity(X, filters, name_base+ '/Mask/Residual_id_3_Down')
    
    Residual_id_3_Down_shortcut = X
    
    Residual_id_3_Down_branched = res_identity(X, filters, name_base+ '/Mask/Residual_id_3_Down_branched')
    
    X = MaxPooling2D((3,3), strides=(2,2), padding='same', name=name_base+ '/Mask/pool_2')(X)
    
    X = res_identity(X, filters, name_base+ '/Mask/Residual_id_2_Down')
    
    Residual_id_2_Down_shortcut = X
    
    Residual_id_2_Down_branched = res_identity(X, filters, name_base+ '/Mask/Residual_id_2_Down_branched')
    
    X = MaxPooling2D((3,3), strides=(2,2), padding='same', name=name_base+ '/Mask/pool_1')(X)
    
    X = res_identity(X, filters, name_base+ '/Mask/Residual_id_1_Down')
    
    X = res_identity(X, filters, name_base+ '/Mask/Residual_id_1_Up')
    
    temp_name1 = name_base+ "/Mask/Interpool_1"
    
    X = Lambda(interpolation, arguments={'ref_tensor': Residual_id_2_Down_shortcut,'name':temp_name1})(X)
                                          
    X = Add(name=base + '/Mask/Add_after_Interpool_1')([X, Residual_id_2_Down_branched])
                                          
    X = res_identity(X, filters, name_base+ '/Mask/Residual_id_2_Up')
    
    temp_name2 = name_base+ "/Mask/Interpool_2"
    
    X = Lambda(interpolation, arguments={'ref_tensor': Residual_id_3_Down_shortcut,'name':temp_name2})(X)
                                          
    X = Add(name=base + '/Mask/Add_after_Interpool_2')([X, Residual_id_3_Down_branched])
                                          
    X = res_identity(X, filters, name_base+ '/Mask/Residual_id_3_Up')
    
    temp_name3 = name_base+ "/Mask/Interpool_3"
    
    X = Lambda(interpolation, arguments={'ref_tensor': X_Trunk,'name':temp_name3})(X)
                                          
    X = BatchNormalization(axis=-1, name=name_base + '/Mask/Interpool_3/bn_1')(X)
                                          
    X = Activation('relu', name=name_base + '/Mask/Interpool_3/relu_1')(X)
                                          
    X = Conv2D(F3, kernel_size=(1,1), strides=(1,1), padding='valid', name=name_base + '/Mask/Interpool_3/conv_1', kernel_initializer=glorot_uniform(seed=0))(X)
    
    X = BatchNormalization(axis=-1, name=name_base + '/Mask/Interpool_3/bn_2')(X)
                                          
    X = Activation('relu', name=name_base + '/Mask/Interpool_3/relu_2')(X)
                                          
    X = Conv2D(F3, kernel_size=(1,1), strides=(1,1), padding='valid', name=name_base + '/Mask/Interpool_3/conv_2', kernel_initializer=glorot_uniform(seed=0))(X)
    
    X = Activation('sigmoid', name=name_base+'/Mask/sigmoid')(X)
    
    X = Multiply(name=name_base+'/Mutiply')([X_Trunk,X])
    
    X = Add(name=name_base+'/Add')([X_Trunk,X])

    X = res_identity(X, filters, name_base+ '/Post_Residual_id')
    
    return X

In [0]:
def Attention_2(X, filters, base):
    
    F1, F2, F3 = filters
    
    name_base = base
    
    X = res_identity(X, filters, name_base+ '/Pre_Residual_id')
    
    X_Trunk = Trunk_block(X, filters, name_base+ '/Trunk')
    
    X = MaxPooling2D((3,3), strides=(2,2), padding='same', name=name_base+ '/Mask/pool_2')(X)
    
    X = res_identity(X, filters, name_base+ '/Mask/Residual_id_2_Down')
    
    Residual_id_2_Down_shortcut = X
    
    Residual_id_2_Down_branched = res_identity(X, filters, name_base+ '/Mask/Residual_id_2_Down_branched')
    
    X = MaxPooling2D((3,3), strides=(2,2), padding='same', name=name_base+ '/Mask/pool_1')(X)
    
    X = res_identity(X, filters, name_base+ '/Mask/Residual_id_1_Down')
                                          
    X = res_identity(X, filters, name_base+ '/Mask/Residual_id_1_Up')
    
    temp_name1 = name_base+ "/Mask/Interpool_1"
    
    X = Lambda(interpolation, arguments={'ref_tensor': Residual_id_2_Down_shortcut,'name':temp_name1})(X)
                                          
    X = Add(name=base + '/Mask/Add_after_Interpool_1')([X, Residual_id_2_Down_branched])
                                          
    X = res_identity(X, filters, name_base+ '/Mask/Residual_id_2_Up')
    
    temp_name2 = name_base+ "/Mask/Interpool_2"
    
    X = Lambda(interpolation, arguments={'ref_tensor': X_Trunk,'name':temp_name2})(X)
    
    X = BatchNormalization(axis=-1, name=name_base + '/Mask/Interpool_2/bn_1')(X)
                                          
    X = Activation('relu', name=name_base + '/Mask/Interpool_2/relu_1')(X)
                                          
    X = Conv2D(F3, kernel_size=(1,1), strides=(1,1), padding='valid', name=name_base + '/Mask/Interpool_2/conv_1', kernel_initializer=glorot_uniform(seed=0))(X)
    
    X = BatchNormalization(axis=-1, name=name_base + '/Mask/Interpool_2/bn_2')(X)
                                          
    X = Activation('relu', name=name_base + '/Mask/Interpool_2/relu_2')(X)
                                          
    X = Conv2D(F3, kernel_size=(1,1), strides=(1,1), padding='valid', name=name_base + '/Mask/Interpool_2/conv_2', kernel_initializer=glorot_uniform(seed=0))(X)
    
    X = Activation('sigmoid', name=name_base+'/Mask/sigmoid')(X)
      
    X = Multiply(name=name_base+'/Mutiply')([X_Trunk,X])
    
    X = Add(name=name_base+'/Add')([X_Trunk,X])

    X = res_identity(X, filters, name_base+ '/Post_Residual_id')
    
    return X

In [0]:
def Attention_3(X, filters, base):
    
    F1, F2, F3 = filters
    
    name_base = base
    
    X = res_identity(X, filters, name_base+ '/Pre_Residual_id')
    
    X_Trunk = Trunk_block(X, filters, name_base+ '/Trunk')
    
    X = MaxPooling2D((3,3), strides=(2,2), padding='same', name=name_base+ '/Mask/pool_1')(X)
    
    X = res_identity(X, filters, name_base+ '/Mask/Residual_id_1_Down')
                                          
    X = res_identity(X, filters, name_base+ '/Mask/Residual_id_1_Up')
    
    temp_name2 = name_base+ "/Mask/Interpool_1"
    
    X = Lambda(interpolation, arguments={'ref_tensor': X_Trunk,'name':temp_name2})(X)
                                          
    X = BatchNormalization(axis=-1, name=name_base + '/Mask/Interpool_2/bn_1')(X)
                                          
    X = Activation('relu', name=name_base + '/Mask/Interpool_2/relu_1')(X)
                                          
    X = Conv2D(F3, kernel_size=(1,1), strides=(1,1), padding='valid', name=name_base + '/Mask/Interpool_2/conv_1', kernel_initializer=glorot_uniform(seed=0))(X)
    
    X = BatchNormalization(axis=-1, name=name_base + '/Mask/Interpool_2/bn_2')(X)
                                          
    X = Activation('relu', name=name_base + '/Mask/Interpool_2/relu_2')(X)
                                          
    X = Conv2D(F3, kernel_size=(1,1), strides=(1,1), padding='valid', name=name_base + '/Mask/Interpool_2/conv_2', kernel_initializer=glorot_uniform(seed=0))(X)
    
    X = Activation('sigmoid', name=name_base+'/Mask/sigmoid')(X)
      
    X = Multiply(name=name_base+'/Mutiply')([X_Trunk,X])
    
    X = Add(name=name_base+'/Add')([X_Trunk,X])

    X = res_identity(X, filters, name_base+ '/Post_Residual_id')
    
    return X

In [0]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

In [0]:
conv_base=InceptionResNetV2(include_top=False, weights='imagenet',input_shape=(299,299,3))

Instructions for updating:
Colocations handled automatically by placer.
219062272/219055592 [==============================] - 7s 0us/step


In [0]:
for layer in conv_base.layers:
    layer.trainable = False

In [0]:
n4=Dropout(0.5)

In [0]:
n2=GlobalAveragePooling2D()

In [0]:
n3=Dense(1, activation='sigmoid', name='my_dense_2')

In [0]:
kfold = StratifiedKFold(n_splits=5, shuffle=False, random_state=7)

In [0]:
cvscores = []

In [0]:
for train, test in kfold.split(x, y):
  trainer = pd.DataFrame(
    {'image_name': x[train],
     'label': y[train],
    })
  tester = pd.DataFrame(
    {'image_name': x[test],
     'label': y[test],
    })
  trainer.reset_index(inplace=True)
  
  trainer.drop(['index'], axis=1,inplace=True)
  
  tester.reset_index(inplace=True)
  
  tester.drop(['index'], axis=1,inplace=True)
  
  datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
  
  train_generator=datagen.flow_from_dataframe(dataframe=trainer, directory="Agg_AMT_Candidates", x_col="image_name", y_col="label", class_mode="binary", target_size=(299,299), batch_size=32,shuffle=False)
  
  datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
  
  test_generator=datagen.flow_from_dataframe(dataframe=tester, directory="Agg_AMT_Candidates", x_col="image_name", y_col="label", class_mode="binary", target_size=(299,299), batch_size=32,shuffle=False)
  
  X = Conv2D(64, (7,7), strides=(2,2), padding='same', name='conv_1', kernel_initializer=glorot_uniform(seed=0))
  
  inp2 = conv_base.input
  out2 = X(conv_base.output)
  out2 = res_conv(out2, [64,64,256], 'Residual_conv_1', 1)
  out2 = Attention_1(out2, [64,64,256], 'Attention_1')
  model2 = Model(inp2, out2)
  inp5 = model2.input
  out5 = n4(model2.output)
  model5 = Model(inp5, out5)
  inp3 = model5.input
  out3 = n2(model5.output)
  model3 = Model(inp3, out3)
  inp4 = model3.input
  out4 = n3(model3.output)
  model4 = Model(inp4, out4)
  
  model4.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
  
  model4.fit_generator(train_generator,steps_per_epoch=100, epochs=1)
  
  scores=model4.evaluate_generator(test_generator, steps=50)
  
  print("%s: %.2f%%" % (model4.metrics_names[1], scores[1]*100))
  
  cvscores.append(scores[1] * 100)

Found 1015 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
Epoch 1/1
100/100 [==============================] - 142s 1s/step - loss: 0.6105 - acc: 0.7646
acc: 74.50%
Found 1015 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
Epoch 1/1
100/100 [==============================] - 131s 1s/step - loss: 0.5661 - acc: 0.7592
acc: 75.76%
Found 1015 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
Epoch 1/1
100/100 [==============================] - 138s 1s/step - loss: 0.5522 - acc: 0.7538
acc: 76.39%
Found 1015 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
Epoch 1/1
100/100 [==============================] - 142s 1s/step - loss: 0.5136 - acc: 0.7798
acc: 68.83%
Found 1016 images belonging to 2 classes.
Found 253 images belonging to 2 classes.
Epoch 1/1
100/100 [==============================] - 148s 1s/step - loss: 0.5734 - acc: 0.7639
acc: 69.34%


In [0]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

72.96% (+/- 3.23%)


In [0]:
for train, test in kfold.split(x, y):
  trainer = pd.DataFrame(
    {'image_name': x[train],
     'label': y[train],
    })
  tester = pd.DataFrame(
    {'image_name': x[test],
     'label': y[test],
    })
  trainer.reset_index(inplace=True)
  
  trainer.drop(['index'], axis=1,inplace=True)
  
  tester.reset_index(inplace=True)
  
  tester.drop(['index'], axis=1,inplace=True)
  
  datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
  
  train_generator=datagen.flow_from_dataframe(dataframe=trainer, directory="Agg_AMT_Candidates", x_col="image_name", y_col="label", class_mode="binary", target_size=(299,299), batch_size=32,shuffle=False)
  
  datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
  
  test_generator=datagen.flow_from_dataframe(dataframe=tester, directory="Agg_AMT_Candidates", x_col="image_name", y_col="label", class_mode="binary", target_size=(299,299), batch_size=32,shuffle=False)
  
  X = Conv2D(64, (7,7), strides=(2,2), padding='same', name='conv_1', kernel_initializer=glorot_uniform(seed=0))
  
  inp2 = conv_base.input
  out2 = X(conv_base.output)
  out2 = res_conv(out2, [64,64,256], 'Residual_conv_1', 1)
  out2 = Attention_1(out2, [64,64,256], 'Attention_1')
  model2 = Model(inp2, out2)
  inp5 = model2.input
  out5 = n4(model2.output)
  model5 = Model(inp5, out5)
  inp3 = model5.input
  out3 = n2(model5.output)
  model3 = Model(inp3, out3)
  inp4 = model3.input
  out4 = n3(model3.output)
  model4 = Model(inp4, out4)
  
  model4.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
  
  model4.fit_generator(train_generator,steps_per_epoch=100, epochs=2)
  
  scores=model4.evaluate_generator(test_generator, steps=50)
  
  print("%s: %.2f%%" % (model4.metrics_names[1], scores[1]*100))
  
  cvscores.append(scores[1] * 100)

Found 1015 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
Epoch 1/2
100/100 [==============================] - 148s 1s/step - loss: 0.5319 - acc: 0.7732
Epoch 2/2
100/100 [==============================] - 112s 1s/step - loss: 0.2639 - acc: 0.8938
acc: 65.43%
Found 1015 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
Epoch 1/2
100/100 [==============================] - 155s 2s/step - loss: 0.5032 - acc: 0.7772
Epoch 2/2
100/100 [==============================] - 111s 1s/step - loss: 0.2443 - acc: 0.8998
acc: 76.89%
Found 1015 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
Epoch 1/2
100/100 [==============================] - 156s 2s/step - loss: 0.5230 - acc: 0.7738
Epoch 2/2
100/100 [==============================] - 113s 1s/step - loss: 0.2719 - acc: 0.8856
acc: 79.22%
Found 1015 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
Epoch 1/2
100/100 [==============================] - 164s 2s/s

In [0]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

73.60% (+/- 4.20%)


Its 74.24%

In [0]:
for train, test in kfold.split(x, y):
  trainer = pd.DataFrame(
    {'image_name': x[train],
     'label': y[train],
    })
  tester = pd.DataFrame(
    {'image_name': x[test],
     'label': y[test],
    })
  trainer.reset_index(inplace=True)
  
  trainer.drop(['index'], axis=1,inplace=True)
  
  tester.reset_index(inplace=True)
  
  tester.drop(['index'], axis=1,inplace=True)
  
  datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
  
  train_generator=datagen.flow_from_dataframe(dataframe=trainer, directory="Agg_AMT_Candidates", x_col="image_name", y_col="label", class_mode="binary", target_size=(299,299), batch_size=32,shuffle=False)
  
  datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
  
  test_generator=datagen.flow_from_dataframe(dataframe=tester, directory="Agg_AMT_Candidates", x_col="image_name", y_col="label", class_mode="binary", target_size=(299,299), batch_size=32,shuffle=False)
  
  X = Conv2D(64, (7,7), strides=(2,2), padding='same', name='conv_1', kernel_initializer=glorot_uniform(seed=0))
  
  inp2 = conv_base.input
  out2 = X(conv_base.output)
  out2 = res_conv(out2, [64,64,256], 'Residual_conv_1', 1)
  out2 = Attention_1(out2, [64,64,256], 'Attention_1')
  model2 = Model(inp2, out2)
  inp5 = model2.input
  out5 = n4(model2.output)
  model5 = Model(inp5, out5)
  inp3 = model5.input
  out3 = n2(model5.output)
  model3 = Model(inp3, out3)
  inp4 = model3.input
  out4 = n3(model3.output)
  model4 = Model(inp4, out4)
  
  model4.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
  
  model4.fit_generator(train_generator,steps_per_epoch=100, epochs=3)
  
  scores=model4.evaluate_generator(test_generator, steps=50)
  
  print("%s: %.2f%%" % (model4.metrics_names[1], scores[1]*100))
  
  cvscores.append(scores[1] * 100)

Found 1015 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
Epoch 1/3
100/100 [==============================] - 168s 2s/step - loss: 0.4914 - acc: 0.7764
Epoch 2/3
100/100 [==============================] - 109s 1s/step - loss: 0.2384 - acc: 0.8999
Epoch 3/3
100/100 [==============================] - 109s 1s/step - loss: 0.1473 - acc: 0.9389
acc: 71.03%
Found 1015 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
Epoch 1/3
100/100 [==============================] - 172s 2s/step - loss: 0.4959 - acc: 0.7814
Epoch 2/3
100/100 [==============================] - 108s 1s/step - loss: 0.2485 - acc: 0.8965
Epoch 3/3
100/100 [==============================] - 108s 1s/step - loss: 0.1570 - acc: 0.9424
acc: 77.77%
Found 1015 images belonging to 2 classes.
Found 254 images belonging to 2 classes.
Epoch 1/3
100/100 [==============================] - 184s 2s/step - loss: 0.5092 - acc: 0.7664
Epoch 2/3
100/100 [==============================] - 108s 1s/s

In [0]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

73.46% (+/- 3.91%)
